## Setup

In [1]:
#r "nuget: Castle.Core"

using System.Collections.Immutable;
using System.Linq.Expressions;
using System.Reflection;
using Castle.DynamicProxy;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Castle.Core, 5.1.1

## Executed and Arg classes

In [2]:
public class Executed
{
  private readonly Func<int, bool> _validate;
  private readonly string _message;

  public Executed(Func<int, bool> validate, string message)
  {
    _validate = validate;
    _message = message;
  }

  public void Verify(int count)
  {
    if (!_validate.Invoke(count))
    {
      throw new Exception($"Executed wrong number of times. Expected: {_message} Executed: {count}");
    }
  }

  public static Executed Once => new (x => x == 1, "Once");
  public static Executed Never => new (x => x == 0, "Never");
  public static Executed AtLeast(int times) => new (x => x >= times, $"At least {times}");
  public static Executed AtMost(int times) => new (x => x <= times, $"At most {times}");
}


In [6]:
public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
  public static T Is<T>(Func<T, bool> matcher) => throw new InvalidOperationException();
}

## MatchInvocation (CastleCore Invocation wrapper) + ProxyGenerator (also Castle Core Wrapper)

In [7]:
public class MockInvocation 
{
  public MockInvocation(IInvocation invocation)
  {
    Arguments = invocation.Arguments.ToImmutableArray();
    Method = invocation.Method;
  }
  
  public MethodInfo Method { get; }
  public ImmutableArray<object> Arguments { get; }
}

public class MockInterceptor : StandardInterceptor
{
  private readonly List<MockInvocation> _invocations = new();

  public IReadOnlyList<MockInvocation> Invocations => _invocations;

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation)
  {
  }
}

In [8]:
public class ProxyFactory
{
  private readonly ProxyGenerator _generator = new ();

  public T GetProxy<T>(IInterceptor interceptor) where T:class => _generator.CreateInterfaceProxyWithoutTarget<T>(interceptor);
}

public static class FactoryProvider 
{
  public static ProxyFactory Factory {get;} = new ProxyFactory();
}

## ExpressionVisitor to Match Args conditions 

In [9]:
public class Visitor : ExpressionVisitor
{
  private static readonly MethodInfo IsMethod = typeof(Arg).GetMethods(BindingFlags.Static | BindingFlags.Public).Single(x => x.Name == "Is");
  public MethodInfo Method { get; private set; }
  public IReadOnlyList<Func<object, bool>> ArgumentValidators { get; private set; }
  protected override Expression VisitMethodCall(MethodCallExpression node)
  {
    Method = node.Method;
    var argumentValidators = new List<Func<object, bool>>();
    foreach (var argument in node.Arguments)
    {
      if (argument.NodeType == ExpressionType.Constant)
      {
        argumentValidators.Add(x => x.Equals(((ConstantExpression)argument).Value));
      }
      else if (argument.NodeType == ExpressionType.Call)
      {
        VisitItExpression(argumentValidators, (MethodCallExpression)argument);
      }
    }

    ArgumentValidators = argumentValidators;
    return node;
  }
  
  private void VisitItExpression(List<Func<object,bool>> argumentValidators, MethodCallExpression argument)
  {
    if (argument.Method.DeclaringType != typeof(Arg))
    {
      throw new InvalidOperationException("Only Arg.Is* is supported");
    }
    else if (argument.Method.IsGenericMethod)
    {
      var genericMethodDefinition = argument.Method.GetGenericMethodDefinition();
      if (genericMethodDefinition == IsMethod)
      {
        argumentValidators.Add(VisitIsMethod(argument));
      }
      // Handle Other Methods...
    }
  }

  private Func<object, bool> VisitIsMethod(MethodCallExpression argument)
  {
    var arg = (LambdaExpression)argument.Arguments.Single(); // We know that Arg.Is has only one argument
    return BuildArgumentValidator(arg.Parameters.Single().Type, arg);
  }

  private Func<object, bool> BuildArgumentValidator(Type castToType, Expression validateExpression)
  {
    var arg = Expression.Parameter(typeof(object));
    var casted = Expression.Convert(arg, castToType);
    var body = Expression.Invoke(validateExpression, casted);
    return Expression.Lambda<Func<object, bool>>(body, arg).Compile();
  }
}

public class MockInvocationMatch
{
  private readonly MethodInfo _method;
  private readonly IReadOnlyList<Func<object, bool>> _argumentValidators;

  public MockInvocationMatch(Expression expression)
  {
    var visitor = new Visitor();
    visitor.Visit(expression);
    (_method, _argumentValidators) = (visitor.Method, visitor.ArgumentValidators);
  }
  public bool IsMatch(MockInvocation invocation) => invocation.Method == _method && invocation.Arguments.Zip(_argumentValidators, (x, y) => y(x)).All(x => x);
}

## Central Mock class with valid Match Method

In [10]:
public class MockObject<T>  where T : class 
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();
  
  public MockObject() 
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationMatch(action);
    times.Verify(_interceptor.Invocations.Count(x => invocationMatch.IsMatch(x)));
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

## Demos

In [11]:
public interface IFoo
{
  void M(int n);
}

var mockObj = new MockObject<IFoo>();
mockObj.Object.M(10);

In [12]:
mockObj.Verify(x => x.M(Arg.Is<int>(n => n == 10)), Executed.Once);

In [13]:
mockObj.Verify(x => x.M(Arg.Is<int>(n => n == 11)), Executed.Once);

Error: System.Exception: Executed wrong number of times. Expected: Once Executed: 0
   at Submission#4.Executed.Verify(Int32 count)
   at Submission#12.MockObject`1.Verify(Expression`1 action, Executed times)
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)